# Transformer Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path

wd = Path(os.path.dirname(os.path.abspath("__file__"))).parent.resolve()
sys.path.append(str(wd))

import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from datetime import datetime, timedelta
from src.models.auction_transformer import AuctionTransformer
from src.models.inference import predict_dataframe

pd.options.display.max_columns = None
pd.options.display.width = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
prediction_time = datetime.strptime("2025-03-20 00:00:00", "%Y-%m-%d %H:%M:%S")

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
        item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

feature_stats = torch.load('../generated/feature_stats.pt')

time_left_mapping = {
    'VERY_LONG': 48,
    'LONG': 12,
    'MEDIUM': 2,
    'SHORT': 0.5
}

In [4]:
from src.data.utils import load_auctions_from_sample

data_dir = '../data/sample/'

df_auctions = load_auctions_from_sample(data_dir, prediction_time)

print("Auctions shape:", df_auctions.shape)
df_auctions.head()

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 99131/99131 [00:01<00:00, 57612.42it/s] 


Auctions shape: (90433, 14)


,id,item_index,bid,buyout,quantity,time_left,context,bonus_lists,modifier_types,modifier_values,first_appearance,last_appearance,current_hours,hours_on_sale
0,1619557849,16874,0.0,2219.81,1,12.0,22,"[1083, 766, 1113, 1111, 421, 1129, 321]",[4],[2462],2025-03-18 04:00:00,2025-03-20 04:00:00,44.0,4.0
1,1618838050,13815,0.0,41006.22,1,2.0,12,"[375, 734, 1161]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0
2,1618591925,16890,0.0,8290.00,1,2.0,60,"[1105, 766, 360, 1129]","[3, 4]","[73, 2832]",2025-03-18 01:00:00,2025-03-20 00:00:00,47.0,0.0
3,1619694628,3062,0.0,1066.23,1,12.0,0,"[767, 373]","[3, 4]","[30, 35]",2025-03-18 05:00:00,2025-03-20 04:00:00,43.0,4.0
4,1620037804,3018,0.0,427.48,1,12.0,0,"[767, 373]","[3, 4]","[30, 35]",2025-03-18 06:00:00,2025-03-20 06:00:00,42.0,6.0


In [31]:
df_auctions[['current_hours', 'hours_on_sale']].describe()

,current_hours,hours_on_sale
count,67008.000000,67008.000000
mean,19.151833,16.134999
std,14.915572,13.320895
min,0.000000,0.000000
25%,6.000000,4.000000
50%,19.000000,15.000000
75%,28.000000,24.000000
max,48.000000,48.000000


In [6]:
model = AuctionTransformer.load_from_checkpoint(
    '../models/auction_transformer_7.2M_128b_wpos_ch_filtered/last-v1.ckpt',
    map_location=device
)

print(f'Number of model parameters: {sum(p.numel() for p in model.parameters())}')
model.eval()
print('Pre-trained Transformer model loaded successfully.')

Number of model parameters: 7208865
Pre-trained Transformer model loaded successfully.


/home/angel/miniconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.1, which is newer than your current Lightning version: v2.5.0.post0


In [8]:
from src.models.inference import predict_dataframe

model = model.to('cuda')
df_auctions = predict_dataframe(model, df_auctions, prediction_time, feature_stats)

print("Mean hours on sale:", df_auctions['hours_on_sale'].mean())
print("Mean prediction:", df_auctions['prediction'].mean())
print("Mean sale probability:", df_auctions['sale_probability'].mean())

df_auctions.head()

  0%|          | 0/15643 [00:00<?, ?it/s]

100%|██████████| 15643/15643 [02:56<00:00, 88.84it/s] 

Skipped 23425 auctions from 15643 item indices
Mean hours on sale: 16.1349988061127
Mean prediction: 21.266601152101245
Mean sale probability: 0.4651507367191592


,id,item_index,bid,buyout,quantity,time_left,context,bonus_lists,modifier_types,modifier_values,first_appearance,last_appearance,current_hours,hours_on_sale,prediction,sale_probability
1,1618838050,13815,0.0,41006.22,1,2.0,12,"[375, 734, 1161]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,15.57,0.535511
3,1619694628,3062,0.0,1066.23,1,12.0,0,"[767, 373]","[3, 4]","[30, 35]",2025-03-18 05:00:00,2025-03-20 04:00:00,43.0,4.0,16.13,0.523779
4,1620037804,3018,0.0,427.48,1,12.0,0,"[767, 373]","[3, 4]","[30, 35]",2025-03-18 06:00:00,2025-03-20 06:00:00,42.0,6.0,18.38,0.478487
6,1620157406,16433,0.0,2010.24,1,12.0,38,[],[4],[2163],2025-03-18 07:00:00,2025-03-20 07:00:00,41.0,7.0,16.69,0.512066
7,1620355747,6186,0.0,825.00,1,12.0,3,"[767, 361]","[3, 4]","[30, 1074]",2025-03-18 09:00:00,2025-03-20 08:00:00,39.0,8.0,16.14,0.523406


In [39]:
df_auctions_12h = df_auctions[df_auctions['current_hours'] <= 0]
df_auctions_12h = df_auctions_12h[df_auctions_12h['time_left'] > 12.0]
len(df_auctions_12h)

mae = mean_absolute_error(df_auctions_12h['hours_on_sale'], df_auctions_12h['prediction'])
print(f"Mean absolute error: {mae}")

Mean absolute error: 8.46449323575072


In [13]:
df_auctions.sample(5)

,id,item_index,bid,buyout,quantity,time_left,context,bonus_lists,modifier_types,modifier_values,first_appearance,last_appearance,current_hours,hours_on_sale,prediction,sale_probability
34639,1623273485,13896,0.0,67876.94,1,2.0,0,"[767, 387]","[3, 4]","[50, 502]",2025-03-19 02:00:00,2025-03-20 01:00:00,22.0,1.0,9.90,0.672317
4996,1618840889,5109,0.0,3399.00,1,2.0,0,[771],"[3, 4]","[30, 20]",2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,14.18,0.566222
75205,1626694562,4321,0.0,7797.46,1,12.0,2,"[767, 357]","[3, 4]","[30, 2884]",2025-03-19 21:00:00,2025-03-20 08:00:00,3.0,8.0,6.13,0.782014
89822,1627368512,5395,0.0,2152.71,1,48.0,13,[],[4],[866],2025-03-20 00:00:00,2025-03-20 23:00:00,0.0,23.0,30.56,0.293612
63810,1625894811,11370,0.0,19389.52,1,12.0,0,[],[4],[860],2025-03-19 17:00:00,2025-03-20 01:00:00,7.0,1.0,3.31,0.875558


In [12]:
# save the dataframe to excel
df_auctions_12h.to_excel('../generated/predictions.xlsx', index=False)

### Binary classification

In [44]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

df_auctions_12h['sold_gt'] = df_auctions_12h['hours_on_sale'] <= 12
df_auctions_12h['sold_pred'] = df_auctions_12h['sale_probability'] >= 0.7

accuracy = accuracy_score(df_auctions_12h['sold_gt'], df_auctions_12h['sold_pred'])
precision = precision_score(df_auctions_12h['sold_gt'], df_auctions_12h['sold_pred'])
recall = recall_score(df_auctions_12h['sold_gt'], df_auctions_12h['sold_pred'])
f1 = f1_score(df_auctions_12h['sold_gt'], df_auctions_12h['sold_pred'])

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")

Accuracy: 0.9583056110002218
Precision: 0.8426966292134831
Recall: 0.30120481927710846
F1 score: 0.4437869822485207


## Error analysis

In [ ]:
columns = [
    'item_name',
    'bid',
    'buyout',
    'quantity',
    'time_left',
    'first_appearance',
    'last_appearance',
    'hours_since_first_appearance',
    'hours_on_sale',
    'prediction',
    'sale_probability'
]

df_error = df_auctions[columns].copy()
df_error['error'] = np.abs(df_error['hours_on_sale'] - df_error['prediction'])
df_error['time_left'] = df_error['time_left'].map(time_left_mapping)

df_error.head()

In [ ]:
plt.boxplot(df_error[['hours_on_sale', 'prediction']])
plt.grid()
plt.show()

In [ ]:
# histogram of hours on sale and prediction
plt.hist(df_error['hours_on_sale'], bins=50, alpha=0.5, label='Hours on sale')
plt.hist(df_error['prediction'], bins=50, alpha=0.5, label='Prediction')
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
plt.hist(df_error[(df_error['time_left'] == 48.0) & (df_error['hours_since_first_appearance'] < 12)]['hours_on_sale'], bins=50, alpha=0.5, label='Hours on sale')
plt.hist(df_error[(df_error['time_left'] == 48.0) & (df_error['hours_since_first_appearance'] < 12)]['prediction'], bins=50, alpha=0.5, label='Prediction')
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
plt.hist(df_error['hours_since_first_appearance'], bins=15)
plt.grid()
plt.show()

We are interested in evaluating the model when the items are recently published, because this will be the main use case for the model

In [ ]:
query = (df_error['hours_since_first_appearance'] <= 12) & (df_error['hours_on_sale'] <= 5) & (df_error['time_left'] == 48.0)
query_df = df_error[query]
print(f"Mean sale probability: {query_df['sale_probability'].mean()}")
print(f"Mean error: {query_df['error'].mean()}")
print(f"Mean hours on sale: {query_df['hours_on_sale'].mean()}")
query_df.sample(10)

In [ ]:
query_df['hours_on_sale'].hist(bins=10)
plt.grid()
plt.show()

In [ ]:
corr_matrix = df_error[['bid', 'buyout', 'quantity', 'time_left', 'hours_since_first_appearance', 'sale_probability',
                        'hours_on_sale', 'prediction', 'error']].corr()

plt.figure(figsize=(10, 8))

sns.heatmap(corr_matrix, 
            annot=True, 
            cmap='coolwarm',
            vmin=-1, vmax=1, 
            center=0,
            fmt='.2f',
            square=True) 

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()

plt.show()

In [18]:
df_error.to_csv('../generated/predictions.csv', index=False)

# Eval validation

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import sys
from tqdm import tqdm
from datetime import datetime
pd.options.mode.chained_assignment = None

from pathlib import Path

repo_root = Path.cwd().parent.resolve()
sys.path.append(str(repo_root))

from sklearn.model_selection import train_test_split
from src.data.auction_dataset import AuctionDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [20]:
pairs = pd.read_csv('../generated/auction_indices.csv')
pairs = pairs[pairs['g_hours_on_sale_max'] < 50]
pairs = pairs[pairs['g_current_hours_max'] < 50]

train_pairs, val_pairs = train_test_split(pairs, test_size=0.15, random_state=42, shuffle=False)

print(f"Before filtering: {len(train_pairs)}")

train_pairs = train_pairs[train_pairs['g_hours_on_sale_len'] <= 32]
val_pairs = val_pairs[val_pairs['g_hours_on_sale_len'] <= 32]

print(f"After filtering: {len(train_pairs)}\n")

val_pairs = val_pairs[val_pairs['record'] < '2025-03-23 00:00:00']

print(f"Train pairs: {len(train_pairs)}")
print(f"Val pairs: {len(val_pairs)}")

Before filtering: 3268330
After filtering: 3221870

Train pairs: 3221870
Val pairs: 489959


In [21]:
val_pairs = val_pairs[val_pairs['record'] == '2025-03-20 00:00:00']
val_pairs.head()

,record,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
3389528,2025-03-20 00:00:00,13815,8,6.88,10.18,1.0,31.0,33.25,17.92,7.0,47.0
3389530,2025-03-20 00:00:00,3062,9,14.67,7.87,4.0,24.0,24.33,16.27,0.0,43.0
3389531,2025-03-20 00:00:00,3018,9,13.11,10.36,3.0,29.0,19.33,17.91,0.0,43.0
3389533,2025-03-20 00:00:00,16433,4,17.75,14.82,3.0,41.0,20.50,14.40,6.0,41.0
3389534,2025-03-20 00:00:00,6186,4,9.50,7.53,2.0,22.0,31.25,8.95,20.0,41.0


In [22]:
import json
import os

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
    item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

feature_stats = torch.load('../generated/feature_stats.pt')

In [23]:
val_pairs.tail()

,record,item_index,g_hours_on_sale_len,g_hours_on_sale_mean,g_hours_on_sale_std,g_hours_on_sale_min,g_hours_on_sale_max,g_current_hours_mean,g_current_hours_std,g_current_hours_min,g_current_hours_max
3405165,2025-03-20 00:00:00,13237,1,31.0,0.0,31.0,31.0,0.0,0.0,0.0,0.0
3405166,2025-03-20 00:00:00,13229,1,47.0,0.0,47.0,47.0,0.0,0.0,0.0,0.0
3405167,2025-03-20 00:00:00,13240,1,47.0,0.0,47.0,47.0,0.0,0.0,0.0,0.0
3405168,2025-03-20 00:00:00,13241,1,8.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0
3405169,2025-03-20 00:00:00,10895,5,24.0,0.0,24.0,24.0,0.0,0.0,0.0,0.0


In [24]:
from src.models.auction_transformer import AuctionTransformer

model = AuctionTransformer.load_from_checkpoint(
    '../models/auction_transformer_7.2M_128b_wpos_ch_filtered/last-v1.ckpt',
    map_location=device
)

print(f'Number of model parameters: {sum(p.numel() for p in model.parameters())}')
model.eval()
print('Pre-trained Transformer model loaded successfully.')

Number of model parameters: 7208865
Pre-trained Transformer model loaded successfully.


/home/angel/miniconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.5.1, which is newer than your current Lightning version: v2.5.0.post0


In [25]:
from src.data.auction_dataset import AuctionDataset
from src.data.utils import collate_auctions

batch_size = 32

val_dataset = AuctionDataset(val_pairs, feature_stats=feature_stats, path='../generated/sequences.h5')
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_auctions) 

Dataset size: 15431


In [30]:
# evaluate the model on the validation set
total_mse = 0
total_mae = 0
total_samples = 0

with torch.no_grad():
    for batch in tqdm(val_dataloader):
        (auctions, item_index, contexts, bonus_lists, modifier_types, modifier_values, current_hours), y = batch

        auctions = auctions.to(device)
        item_index = item_index.to(device)
        contexts = contexts.to(device)
        bonus_lists = bonus_lists.to(device)
        modifier_types = modifier_types.to(device)
        modifier_values = modifier_values.to(device)
        current_hours = current_hours.to(device)
        y = y.to(device)

        model.eval()
        y_hat = model((auctions, item_index, contexts, bonus_lists, modifier_types, modifier_values))

        mask = (item_index != 0).float().unsqueeze(-1)
        current_hours_mask = (current_hours <= 12.0).float().unsqueeze(-1)
        mask = mask * current_hours_mask
        
        mse = torch.nn.functional.mse_loss(y_hat * mask, y.unsqueeze(2) * mask) / mask.sum()
        mae = torch.nn.functional.l1_loss(
            y_hat * mask * 48.0,
            y.unsqueeze(2) * mask * 48.0,
            reduction='sum'
        ) / mask.sum()
        
        total_mse += mse.item() * mask.sum()
        total_mae += mae.item() * mask.sum()
        total_samples += mask.sum()

avg_mse = total_mse / total_samples
avg_mae = total_mae / total_samples
print(f'Validation MSE: {avg_mse}')
print(f'Validation MAE: {avg_mae}')

100%|██████████| 483/483 [00:24<00:00, 19.34it/s]

Validation MSE: 0.00010996170021826401
Validation MAE: 6.773960113525391


In [27]:
from src.models.inference import predict_dataframe

predict_dataframe(model, df_auctions[df_auctions['item_index'] == 13815], prediction_time, feature_stats)

100%|██████████| 1/1 [00:00<00:00, 54.02it/s]

Skipped 0 auctions from 1 item indices


,id,item_index,bid,buyout,quantity,time_left,context,bonus_lists,modifier_types,modifier_values,first_appearance,last_appearance,current_hours,hours_on_sale,prediction,sale_probability
1,1618838050,13815,0.00,41006.22,1,2.0,12,"[375, 734, 1161]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,15.57,0.535511
2243,1618838201,13815,0.00,41006.22,1,2.0,12,"[369, 734, 803]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,15.02,0.547523
3234,1618838288,13815,0.00,41006.22,1,2.0,12,"[388, 734, 801]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,15.07,0.546404
7549,1618838338,13815,0.00,41006.22,1,2.0,12,"[381, 734, 800]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,14.87,0.550959
8992,1618838116,13815,0.00,41006.22,1,2.0,12,"[387, 734, 802]",[4],[708],2025-03-18 01:00:00,2025-03-20 01:00:00,47.0,1.0,14.90,0.550138
44993,1624653045,13815,67499.25,89999.00,1,48.0,12,"[396, 734]",[4],[708],2025-03-19 08:00:00,2025-03-21 07:00:00,16.0,31.0,31.93,0.277958
58513,1625590278,13815,0.00,7499.90,1,48.0,12,"[359, 734]",[4],[708],2025-03-19 16:00:00,2025-03-20 15:00:00,8.0,15.0,43.15,0.177235
63736,1625891920,13815,0.00,7499.89,1,12.0,12,"[366, 734]",[4],[708],2025-03-19 17:00:00,2025-03-20 04:00:00,7.0,4.0,3.09,0.883621
